In [2]:
import torch
import numpy as np
from models import MobileNetMini
from datasets import train_data, augment_data, AugDataset, get_loaders
from utils import train, test
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import ConcatDataset

In [3]:
len(train_data)

50000

In [4]:
# augment train data
aug_train = augment_data(train_data)
aug_dataset = AugDataset(*aug_train)
train_data = ConcatDataset((train_data, aug_dataset))

print(len(train_data))

Augmenting train data...


100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:14<00:00, 3345.02it/s]

150000


In [5]:
train_data

In [6]:
batch_size = 64
data_loaders, data_sizes = get_loaders(train_data, batch_size=batch_size, val_size=0.1)

In [7]:
data_loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x20eefe2c550>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x20eef946bf0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x20eefafba30>}

In [8]:
data_sizes

{'train': 135000, 'val': 15000, 'test': 10000}

In [9]:
cuda = torch.cuda.is_available()

if cuda:
    print("CUDA is available...")
else:
    print("CUDA is not available!")

CUDA is available...


In [10]:
# instantiate model
model = MobileNetMini()
if cuda:
    model.cuda()

lr = 1e-2
decay = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=decay)
criterion = torch.nn.CrossEntropyLoss()

In [11]:
# tensorboard logs
run = "run1"
writer =  SummaryWriter(f'logs/{run}')
model_path = f"./models/{run}/"
Path(model_path).mkdir(exist_ok=True)

In [12]:
initial_epochs = 0
n_epochs = 30

train(model, data_loaders=data_loaders, data_sizes=data_sizes,
        optimizer=optimizer, criterion=criterion, epochs=n_epochs,
         model_path=model_path, writer=writer, initial_epochs=initial_epochs)

100%|███████████████████████████████████████████████████████████████████████████████| 235/235 [00:02<00:00, 114.57it/s]


Epoch 1/30: loss- 1.388, acc- 0.481, val_loss- 1.424, val_acc- 0.522
val_loss decreased from inf to 1.4241. saving model ...


100%|███████████████████████████████████████████████████████████████████████████████| 235/235 [00:02<00:00, 111.97it/s]


Epoch 2/30: loss- 1.010, acc- 0.635, val_loss- 1.393, val_acc- 0.535
val_loss decreased from 1.4241 to 1.3935. saving model ...


100%|███████████████████████████████████████████████████████████████████████████████| 235/235 [00:02<00:00, 114.83it/s]


...

Epoch 28/30: loss- 0.553, acc- 0.805, val_loss- 0.561, val_acc- 0.803
val_loss decreased from 0.5952 to 0.5611. saving model ...


100%|███████████████████████████████████████████████████████████████████████████████| 235/235 [00:02<00:00, 108.56it/s]


Epoch 29/30: loss- 0.548, acc- 0.807, val_loss- 0.583, val_acc- 0.796


100%|███████████████████████████████████████████████████████████████████████████████| 235/235 [00:02<00:00, 109.89it/s]

Epoch 30/30: loss- 0.545, acc- 0.808, val_loss- 0.605, val_acc- 0.788


In [13]:
best_model = r"./models/run1/model.28-0.5611.pt"
model.load_state_dict(torch.load(best_model))

<All keys matched successfully>

In [14]:
test(model, data_loaders=data_loaders, writer=writer)

100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:03<00:00, 39.63it/s]

Test Accuracy of airplane: 75.40%
Test Accuracy of automobile: 92.00%
Test Accuracy of bird: 79.70%
Test Accuracy of cat: 65.30%
Test Accuracy of deer: 76.60%
Test Accuracy of dog: 68.00%
Test Accuracy of frog: 87.60%
Test Accuracy of horse: 80.40%
Test Accuracy of ship: 90.90%
Test Accuracy of truck: 83.80%
Test Accuracy (Overall): 79.97%
